# Lab 5: Supervised Classification

In this lab, we will be reviewing two common classification methods and learn how to leverage these Machine Learning (ML) techniques with Google Earth Engine data. As per usual, we will be leveraging Dr. Qiusheng Wu's [Geemap](https://geemap.org/) package.

There are 4 Conceptual Questions, which will account for 40% of your lab grade. The 2 Challenge Questions will account for 60%.

# Requirements

Everytime you open a new notebook (or your kernel disconnects) you will need to run these requirements before you are able to produce any maps.

The 'pip install' lines are commented out becuase most everyone should already have these libraries installed on your devices, but in case you do not, uncomment and run the first two lines.

In [ ]:
# ! pip install geemap
# ! pip install pygis

import geemap
import ee
from geemap import basemaps

ee.Authenticate()
ee.Initialize(project='utk-dryver2206') # Replace 'YOUR_PROJECT_NAME' with the name of your earth engine cloud project

# Review Lab 4

In our last lab, we learned how to create custom spectral indices using Sentinel-2 Imagery. Here's two examples of code that would achieve the goals of each challenge question.

### Part 1 Review

Calculate three (3) indices of the five that we used in this lab (NDVI, NDWI, NDTI, SAVI, or NBR) for a location of your choosing. (You will need to filterBounds using a point, as we did in each example.)

In [ ]:
# Define the point of interest in Venice, Italy
point = ee.Geometry.Point(11.99, 45.55)

# Load Sentinel-2 imagery
sentinel2 = sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(point) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()


def calculate_ndvi(image):
  """Calculates NDVI from a Sentinel-2 image."""
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def calculate_ndwi(image):
  """Calculates NDWI from a Sentinel-2 image."""
  ndwi = image.normalizedDifference(['B3', 'B11']).rename('NDWI')
  return image.addBands(ndwi)

def calculate_savi(image):
  """Calculates SAVI from a Sentinel-2 image."""
  nir = image.select('B8')
  red = image.select('B4')
  L = 0.5  # Soil adjustment factor
  savi = image.expression(
      '(NIR - RED) / (NIR + RED + L) * (1 + L)',
      {'NIR': nir, 'RED': red, 'L': L}
  ).rename('SAVI')
  return image.addBands(savi)


sentinel2_with_indices = calculate_ndvi(sentinel2) \
    .addBands(calculate_ndwi(sentinel2)) \
    .addBands(calculate_savi(sentinel2))


Map = geemap.Map()
Map.centerObject(point, 11)
Map.addLayer(sentinel2_with_indices.select('NDVI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
Map.addLayer(sentinel2_with_indices.select('NDWI'), {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')
Map.addLayer(sentinel2_with_indices.select('SAVI'), {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'green']}, 'SAVI')

Map


Map(center=[45.55, 11.99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

### Part 2 Review

Calculate and display an image using an index that was not used in this lab. (Some examples include NDSI, BSI, NDBI - [Esri](https://pro.arcgis.com/en/pro-app/latest/arcpy/spatial-analyst/bai.htm) has a good inventory of some options.)

In [ ]:
# Define a point in New York City
point = ee.Geometry.Point([-74.0060, 40.7128])  # Example: Times Square

# Import Sentinel-2 surface reflectance data
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Filter by location and date
image = sentinel \
    .filterBounds(point) \
    .filterDate('2023-01-01', '2023-12-31') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

# Function to calculate NDBI
def calculate_ndbi(image):
  nir = image.select('B8')  # Near-infrared band
  swir = image.select('B11')  # Short-wave infrared band
  ndbi = nir.subtract(swir).divide(nir.add(swir)).rename('NDBI')
  return image.addBands(ndbi)

# Calculate NDBI
image_with_ndbi = calculate_ndbi(image)

# Display the NDBI on a map (optional)
Map = geemap.Map()
Map.centerObject(point, 12)
Map.addLayer(image_with_ndbi.select('NDBI'), {'min': -0.5, 'max': 0.5, 'palette': ['red', 'brown', 'white', 'blue']}, 'NDBI')
Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

# Graded Section

## Conceptual Questions (40%)

### Question 1 (10 Points)

In this lab, we are using Landsat imagery for the first time. What about Landsat's temporal range may make this more useful than Sentinel for our training data? (Think about when Landsat started versus Sentinel.)

### Question 2 (10 Points)

Why is it important for us to make sure the Landsat data we use for training is within the same year as our NLCD data? For example, we used NLCD 2016 data and filtered our Landsat imagery for 2016.

### Question 3 (10 Points)

Analyze the classified image we produced in Part 1 with the CART method. Where does it seem our model did well at predicting land cover? Where did it misrepresent what's happening on the ground? What might explain this difference?

### Question 4 (10 Points)

Describe two scenarios: one where using a buffer to define a training sample area would be more appropriate and one where a custom ROI would be more appropriate.

## Challenge Questions (60%)

### Challenge 1 (30 Points)

Using the Custom ROI method explained in Part 2, define a new sample area and create a classified image with the CART function.

### Challenge 2 (30 Points)

In this lab, we used buffers and custom ROIs to define a training area. However, it may be useful to know how to define your area using a feature collection, such as the TIGER Census dataset. Conduct either a CART or RF analysis using a US county of your choosing for your training data.

Note: Since the NLCD is US-based, it would be easier to refrain from using an area outside of the United States for your training data. However, if you would like to give it a try, let me know and we can work it out!  

# Section 1. Classification and Regression Tree (CART) with Point & Buffer

This example is meant to show not only how to use the built-in CART function, but also how choosing a sample area can impact your end result. We will use a buffer analysis to choose a portion of the Rocky Mountains that will inform our classification of evergreen forests.

### 1.1 Gather Training Data

In [ ]:
# Create an interactive map.
cart = geemap.Map()

# Define training sample region with buffer
region = ee.Geometry.Point([-115.4439, 37.7538]).buffer(10000)

# Load the Landsat 9 image collection.
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2016-01-01', '2016-03-31') \
    .filterBounds(region) \
    .sort('CLOUD_COVER') \
    .first()

# Create a median composite of the collection.
# image = collection.first()

'''

Landsat is weird. They require us to apply scaling factors on the images in
order to get the correct values and view them properly. Scaling factors may look
different when using different versions of Landsat. Due to this confusing aspect,
I've refrained from including Landsat imagery in our labs. However, this example
necessitates the use of Landsat imagery. The two lines below can be applied to
any Landsat 8 or 9 imagery.

'''

# Apply scaling factors to the image.
def apply_scale_factors(image):
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermalBands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

dataset = apply_scale_factors(image)

# Specify visualization parameters.
vis_natural = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "min": 0.0,
    "max": 0.3,
}
vis_nir = {
    "bands": ["SR_B5", "SR_B4", "SR_B3"],
    "min": 0.0,
    "max": 0.3,
}

# Add data layers to the map.
cart.addLayer(dataset, vis_natural, "True color (432)")
cart.addLayer(dataset, vis_nir, "Color infrared (543)")
cart.centerObject(region)

# Display the map.
cart

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define training sample region with buffer
region = ee.Geometry.Point([-115.4439, 37.7538]).buffer(10000)

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training


![](https://i.imgur.com/7QoRXxu.png)

In [ ]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(region)
cart.addLayer(nlcd, {}, "NLCD")
cart.centerObject(nlcd)
cart

EEException: Image.load: Image asset 'USGS/NLCD/NLCD2021' not found (does not exist or caller does not have access).

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False if you want to ignore geometries
    }
)

cart.addLayer(points, {}, "training", False)

In [ ]:
# Display number of points used for training
print('Training Points: ', points.size().getInfo())

# Display point metadata
print(points.first().getInfo())

Training Points:  5000
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-115.38412105321758, 37.79212352551664]}, 'id': '0', 'properties': {'landcover': 52}}


### 1.2 Train Classifier

In [ ]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

### 1.3 Classify Image

In [ ]:
# Load the Landsat 9 image collection.
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2016-01-01', '2016-03-31')

# Create a median composite of the collection.
image = collection.median()

# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
cart.addLayer(result.randomVisualizer(), {}, "classified")
cart

Map(center=[37.75381726998074, -115.44370521047934], controls=(WidgetControl(options=['position', 'transparent…

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [ ]:
# Pull NLCD Class Values
class_values = nlcd.get("landcover_class_values").getInfo()
print('Class Values: ', class_values)

# Pull NLCD Class Color Codes
class_palette = nlcd.get("landcover_class_palette").getInfo()
print('Class Pallette: ', class_palette)

Class Values:  [11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]
Class Pallette:  ['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [ ]:
# Apply Class Values and Color Codes
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

cart.addLayer(landcover, {}, "Land cover")
print("Change layer opacity:")
cluster_layer = cart.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [ ]:
# Add Legend to Map
cart.add_legend(builtin_legend="NLCD")

# Section 2. Random Forest (RF) Classification with Custom ROI

RF classification is the most common method you will see in remote sensing ML applications. There are many different ways to leverage this technology, but in this section, we will draw a custom Region of Interest (ROI) to define our sample area.

Similar to Section 1, the region we select will dramatically impact the results of our analysis.

### 1.1 Gather Training Data

In [ ]:
rf = geemap.Map()
rf

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
rf_region = rf.user_roi

In [ ]:
# Load the Landsat 9 image collection.
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2016-01-01', '2016-12-31')


image = collection.median()

# # Create a median composite of the collection.
# image = collection.median()

# Apply scaling factors to the image.
def apply_scale_factors(image):
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermalBands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

dataset = apply_scale_factors(image)

# Specify visualization parameters.
vis_natural = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "min": 0.0,
    "max": 0.3,
}
vis_nir = {
    "bands": ["SR_B5", "SR_B4", "SR_B3"],
    "min": 0.0,
    "max": 0.3,
}

# Add data layers to the map.
rf.addLayer(dataset, vis_natural, "True color (432)")
rf.addLayer(dataset, vis_nir, "Color infrared (543)")

# Display the map.
rf

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(rf_region)
rf.addLayer(nlcd, {}, "NLCD")
rf.centerObject(nlcd)

EEException: Parameter 'geometry' is required.

In [ ]:
# Make the training dataset.
rf_points = nlcd.sample(
    **{
        "region": rf_region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

rf.addLayer(rf_points, {}, "training", False)

In [ ]:
# Display number of points used for training
print('Training Points: ', points.size().getInfo())

# Display point metadata
print(points.first().getInfo())

### 1.2 Train Classifier

In [ ]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": rf_points, "properties": [label], "scale": 30}
)

# Train a 10-tree random forest classifier from the training sample.
trained = ee.Classifier.smileRandomForest(10).train(
    features=training,
    classProperty=label,
    inputProperties=bands
)

### 1.3 Classify New Image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
rf.addLayer(result.randomVisualizer(), {}, "classified")
rf

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [ ]:
# Pull NLCD Class Values
class_values = nlcd.get("landcover_class_values").getInfo()
print('Class Values: ', class_values)

# Pull NLCD Class Color Codes
class_palette = nlcd.get("landcover_class_palette").getInfo()
print('Class Pallette: ', class_palette)

In [ ]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

rf.addLayer(landcover, {}, "Land cover")
print("Change layer opacity:")
cluster_layer = rf.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

In [ ]:
rf.add_legend(builtin_legend="NLCD")